In [ ]:
import sys
import csv
import math
import operator
import os
from os.path import expanduser
import wget
import shutil

import numpy as np
import pandas as pd
import gensim.models
import gensim.models.word2vec as w2v

from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from scipy.sparse import csr_matrix


MIMIC_3_DIR = "./ori_mimic3_data/mimic3"
DATA_DIR = "./ori_mimic3_data"
SAVE_DIR = "./processed_data"
PAD_CHAR = "**PAD**"
EMBEDDING_SIZE = 100
MAX_LENGTH = 2500

In [ ]:
Y = 'full' #use all available labels in the dataset for prediction
notes_file = '%s/NOTEEVENTS.csv' % MIMIC_3_DIR # raw note events downloaded from MIMIC-III
vocab_size = 'full' #don't limit the vocab size to a specific number
vocab_min = 3 #discard tokens appearing in fewer than this many documents

In [ ]:
dfproc = pd.read_csv('%s/PROCEDURES_ICD.csv' % MIMIC_3_DIR)
dfdiag = pd.read_csv('%s/DIAGNOSES_ICD.csv' % MIMIC_3_DIR)

In [ ]:
def reformat(code, is_diag):
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

In [ ]:
dfdiag['code'] = dfdiag.apply(lambda row: str(reformat(str(row[4]), True)), axis=1)
dfproc['code'] = dfproc.apply(lambda row: str(reformat(str(row[4]), False)), axis=1)

In [ ]:
dfcodes = pd.concat([dfdiag, dfproc])

In [ ]:
dfcodes.to_csv('%s/ALL_CODES.csv' % SAVE_DIR, index=False,
               columns=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'code'],
               header=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'])

In [ ]:
#In the full dataset (not just discharge summaries)
df = pd.read_csv('%s/ALL_CODES.csv' % SAVE_DIR, dtype={"ICD9_CODE": str})
len(df['ICD9_CODE'].unique())

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

def write_discharge_summaries(out_file):
    notes_file = '%s/NOTEEVENTS.csv' % (MIMIC_3_DIR)
    print("processing notes file")
    with open(notes_file, 'r') as csvfile:
        with open(out_file, 'w') as outfile:
            print("writing to %s" % (out_file))
            outfile.write(','.join(['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'TEXT']) + '\n')
            notereader = csv.reader(csvfile)
            #header
            next(notereader)
            i = 0
            for line in tqdm(notereader):
                subj = int(line[1])
                category = line[6]
                if category == "Discharge summary":
                    note = line[10]
                    #tokenize, lowercase and remove numerics
                    tokens = [t.lower() for t in tokenizer.tokenize(note) if not t.isnumeric()]
                    text = '"' + ' '.join(tokens) + '"'
                    outfile.write(','.join([line[1], line[2], line[4], text]) + '\n')
                i += 1
    return out_file

In [ ]:
disch_full_file = write_discharge_summaries(out_file="%s/disch_full.csv" % SAVE_DIR)

In [ ]:
df = pd.read_csv('%s/disch_full.csv' % SAVE_DIR)

In [ ]:
len(df['HADM_ID'].unique())

In [ ]:
#Tokens and types
types = set()
num_tok = 0
for row in df.itertuples():
    for w in row[4].split():
        types.add(w)
        num_tok += 1

In [ ]:
print("Num types", len(types))
print("Num tokens", str(num_tok))

In [ ]:
#Let's sort by SUBJECT_ID and HADM_ID to make a correspondence with the MIMIC-3 label file
df = df.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [ ]:
#Sort the label file by the same
dfl = pd.read_csv('%s/ALL_CODES.csv' % SAVE_DIR)
dfl = dfl.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [ ]:
len(df['HADM_ID'].unique()), len(dfl['HADM_ID'].unique())

In [ ]:
#Let's filter out these HADM_ID's
hadm_ids = set(df['HADM_ID'])
with open('%s/ALL_CODES.csv' % SAVE_DIR, 'r') as lf:
    with open('%s/ALL_CODES_filtered.csv' % SAVE_DIR, 'w') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'ADMITTIME', 'DISCHTIME'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[2])
            #print(hadm_id)
            #break
            if hadm_id in hadm_ids:
                w.writerow(row[1:3] + [row[-1], '', ''])
dfl = pd.read_csv('%s/ALL_CODES_filtered.csv' % SAVE_DIR, index_col=None)
len(dfl['HADM_ID'].unique())

In [ ]:
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

def concat_data(labelsfile, notes_file):
    with open(labelsfile, 'r') as lf:
        print("CONCATENATING")
        with open(notes_file, 'r') as notesfile:
            outfilename = '%s/notes_labeled.csv' % SAVE_DIR
            with open(outfilename, 'w') as outfile:
                w = csv.writer(outfile)
                w.writerow(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS'])

                labels_gen = next_labels(lf)
                notes_gen = next_notes(notesfile)

                for i, (subj_id, text, hadm_id) in enumerate(notes_gen):
                    if i % 10000 == 0:
                        print(str(i) + " done")
                    cur_subj, cur_labels, cur_hadm = next(labels_gen)

                    if cur_hadm == hadm_id:
                        w.writerow([subj_id, str(hadm_id), text, ';'.join(cur_labels)])
                    else:
                        print("couldn't find matching hadm_id. data is probably not sorted correctly")
                        break
                    
    return outfilename

def split_data(labeledfile, base_name):
    print("SPLITTING")
    #create and write headers for train, dev, test
    train_name = '%s_train_split.csv' % (base_name)
    dev_name = '%s_dev_split.csv' % (base_name)
    test_name = '%s_test_split.csv' % (base_name)
    train_file = open(train_name, 'w')
    dev_file = open(dev_name, 'w')
    test_file = open(test_name, 'w')
    train_file.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")
    dev_file.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")
    test_file.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")

    hadm_ids = {}

    #read in train, dev, test splits
    for splt in ['train', 'dev', 'test']:
        hadm_ids[splt] = set()
        with open('%s/%s_full_hadm_ids.csv' % (MIMIC_3_DIR, splt), 'r') as f:
            for line in f:
                hadm_ids[splt].add(line.rstrip())

    with open(labeledfile, 'r') as lf:
        reader = csv.reader(lf)
        next(reader)
        i = 0
        cur_hadm = 0
        for row in reader:
            #filter text, write to file according to train/dev/test split
            if i % 10000 == 0:
                print(str(i) + " read")

            hadm_id = row[1]

            if hadm_id in hadm_ids['train']:
                train_file.write(','.join(row) + "\n")
            elif hadm_id in hadm_ids['dev']:
                dev_file.write(','.join(row) + "\n")
            elif hadm_id in hadm_ids['test']:
                test_file.write(','.join(row) + "\n")

            i += 1

        train_file.close()
        dev_file.close()
        test_file.close()
    return train_name, dev_name, test_name
    
def next_labels(labelsfile):
    labels_reader = csv.reader(labelsfile)
    #header
    next(labels_reader)

    first_label_line = next(labels_reader)

    cur_subj = int(first_label_line[0])
    cur_hadm = int(first_label_line[1])
    cur_labels = [first_label_line[2]]

    for row in labels_reader:
        subj_id = int(row[0])
        hadm_id = int(row[1])
        code = row[2]
        #keep reading until you hit a new hadm id
        if hadm_id != cur_hadm or subj_id != cur_subj:
            yield cur_subj, cur_labels, cur_hadm
            cur_labels = [code]
            cur_subj = subj_id
            cur_hadm = hadm_id
        else:
            #add to the labels and move on
            cur_labels.append(code)
    yield cur_subj, cur_labels, cur_hadm

def next_notes(notesfile):
    nr = csv.reader(notesfile)
    #header
    next(nr)

    first_note = next(nr)

    cur_subj = int(first_note[0])
    cur_hadm = int(first_note[1])
    cur_text = first_note[3]
    
    for row in nr:
        subj_id = int(row[0])
        hadm_id = int(row[1])
        text = row[3]
        #keep reading until you hit a new hadm id
        if hadm_id != cur_hadm or subj_id != cur_subj:
            yield cur_subj, cur_text, cur_hadm
            cur_text = text
            cur_subj = subj_id
            cur_hadm = hadm_id
        else:
            #concatenate to the discharge summary and move on
            cur_text += " " + text
    yield cur_subj, cur_text, cur_hadm


In [ ]:
#we still need to sort it by HADM_ID
dfl = dfl.sort_values(['SUBJECT_ID', 'HADM_ID'])
dfl.to_csv('%s/ALL_CODES_filtered.csv' % SAVE_DIR, index=False)

In [ ]:
#Now let's append each instance with all of its codes
#this is pretty non-trivial so let's use this script I wrote, which requires the notes to be written to file
sorted_file = '%s/disch_full.csv' % SAVE_DIR
df.to_csv(sorted_file, index=False)
labeled = concat_data('%s/ALL_CODES_filtered.csv' % SAVE_DIR, sorted_file)

In [ ]:
#name of the file we just made
print(labeled)

In [ ]:
dfnl = pd.read_csv(labeled)
#Tokens and types
types = set()
num_tok = 0
for row in dfnl.itertuples():
    for w in row[3].split():
        types.add(w)
        num_tok += 1
print("num types", len(types), "num tokens", num_tok)

In [ ]:
len(dfnl['HADM_ID'].unique())

In [ ]:
fname = '%s/notes_labeled.csv' % SAVE_DIR
base_name = "%s/disch" % SAVE_DIR #for output
tr, dv, te = split_data(fname, base_name=base_name)

In [ ]:
def build_vocab(vocab_min, infile, vocab_filename):
    with open(infile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        #header
        next(reader)

        note_numwords = []
        #indices where notes start
        note_inds = [0]
        #indices of discovered words
        indices = []
        #holds a bunch of ones
        data = []
        #keep track of discovered words
        vocab = {}
        #build lookup table for terms
        num2term = {}
        #preallocate array to hold number of notes each term appears in
        note_occur = np.zeros(400000, dtype=int)
        i = 0
        for row in reader:
            text = row[2]
            numwords = 0
            for term in text.split():
                #put term in vocab if it's not there. else, get the index
                index = vocab.setdefault(term, len(vocab))
                indices.append(index)
                num2term[index] = term
                data.append(1)
                numwords += 1
            #record where the next note starts
            note_inds.append(len(indices))
            indset = set(indices[note_inds[-2]:note_inds[-1]])
            #go thru all the word indices you just added, and add to the note occurrence count for each of them
            for ind in indset:
                note_occur[ind] += 1
            note_numwords.append(numwords)
            i += 1
        #clip trailing zeros
        note_occur = note_occur[note_occur>0]

        #turn vocab into a list so indexing doesn't get fd up when we drop rows
        vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

        #1. create sparse document matrix
        C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
        #also need the numwords array to be a sparse matrix
        note_numwords = csr_matrix(1. / np.array(note_numwords))
        
        #2. remove rows with less than 3 total occurrences
        #inds holds indices of rows corresponding to terms that occur in < 3 documents
        inds = np.nonzero(note_occur >= vocab_min)[0]
        print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
        #drop those rows
        C = C[inds,:]
        note_occur = note_occur[inds]
        vocab_list = vocab_list[inds]

        print("writing output")
        with open(vocab_filename, 'w') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")

In [ ]:
vocab_min = 3
vname = '%s/vocab.csv' % SAVE_DIR
build_vocab(vocab_min, tr, vname)

In [ ]:
for splt in ['train', 'dev', 'test']:
    filename = '%s/disch_%s_split.csv' % (SAVE_DIR, splt)
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_full.csv' % (SAVE_DIR, splt), index=False)

In [ ]:
class ProcessedIter(object):

    def __init__(self, Y, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename) as f:
            r = csv.reader(f)
            next(r)
            for row in r:
                yield (row[3].split())

def word_embeddings(Y, notes_file, embedding_size, min_count, epochs):
    modelname = "processed_%s.w2v" % (Y)
    sentences = ProcessedIter(Y, notes_file)

    model = w2v.Word2Vec(vector_size=embedding_size, min_count=min_count, workers=4, sentences=sentences, epochs = epochs)
    print("building word2vec vocab on %s..." % (notes_file))
    
    model.build_vocab(sentences)
    print("training...")
    model.train(sentences, total_examples=model.corpus_count, epochs=epochs)
    out_file = '/'.join(notes_file.split('/')[:-1] + [modelname])
    print("writing embeddings to %s" % (out_file))
    model.save(out_file)
    return out_file

In [ ]:
w2v_file = word_embeddings('full', '%s/disch_full.csv' % SAVE_DIR, 100, 0, 5)

In [ ]:
def gensim_to_embeddings(wv_file, vocab_file, Y, outfile=None):
    model = gensim.models.Word2Vec.load(wv_file)
    wv = model.wv
    #free up memory
    del model

    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}

    W, words = build_matrix(ind2w, wv)

    if outfile is None:
        outfile = wv_file.replace('.w2v', '.embed')

    #smash that save button
    save_embeddings(W, words, outfile)

def build_matrix(ind2w, wv):
    W = np.zeros((len(ind2w)+1, len(wv.word_vec(wv.index_to_key[0])) ))
    words = [PAD_CHAR]
    W[0][:] = np.zeros(len(wv.word_vec(wv.index_to_key[0])))
    for idx, word in tqdm(ind2w.items()):
        if idx >= W.shape[0]:
            break    
        W[idx][:] = wv.word_vec(word)
        words.append(word)
    return W, words

def save_embeddings(W, words, outfile):
    with open(outfile, 'w') as o:
        #pad token already included
        for i in range(len(words)):
            line = [words[i]]
            line.extend([str(d) for d in W[i]])
            o.write(" ".join(line) + "\n")

def load_embeddings(embed_file):
    #also normalizes the embeddings
    W = []
    with open(embed_file) as ef:
        for line in ef:
            line = line.rstrip().split()
            vec = np.array(line[1:]).astype(np.float)
            vec = vec / float(np.linalg.norm(vec) + 1e-6)
            W.append(vec)
        #UNK embedding, gaussian randomly initialized 
        print("adding unk embedding")
        vec = np.random.randn(len(W[-1]))
        vec = vec / float(np.linalg.norm(vec) + 1e-6)
        W.append(vec)
    W = np.array(W)
    return W

In [ ]:
gensim_to_embeddings('%s/processed_full.w2v' % SAVE_DIR, '%s/vocab.csv' % SAVE_DIR, Y)

In [ ]:
DATA_DIR = "./ori_mimic3_data"
def load_code_descriptions():
    desc_dict = defaultdict(str)
    with open("%s/D_ICD_DIAGNOSES.csv" % (DATA_DIR), 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            desc_dict[reformat(code, True)] = desc
    with open("%s/D_ICD_PROCEDURES.csv" % (DATA_DIR), 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            if code not in desc_dict.keys():
                desc_dict[reformat(code, False)] = desc
    with open('%s/ICD9_descriptions' % DATA_DIR, 'r') as labelfile:
        for i,row in enumerate(labelfile):
            row = row.rstrip().split()
            code = row[0]
            if code not in desc_dict.keys():
                desc_dict[code] = ' '.join(row[1:])
    return desc_dict

def vocab_index_descriptions_ref(vocab_file, desc_file, vectors_file):
    #load lookups
    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}
    w2ind = {w:i for i,w in ind2w.items()}
    desc_dict = load_code_descriptions()
        
    tokenizer = RegexpTokenizer(r'\w+')

    with open(vectors_file, 'w') as of:
        with open(desc_file, 'w') as desc_f:
            vw = csv.writer(of, delimiter=' ')
            vw.writerow(["CODE", "VECTOR"])

            dw = csv.writer(desc_f, delimiter=' ')
            dw.writerow(["CODE", "WORD"])

            for code, desc in tqdm(desc_dict.items()):
                #same preprocessing steps as in get_discharge_summaries
                tokens = [t.lower() for t in tokenizer.tokenize(desc) if not t.isnumeric()]
                inds = [w2ind[t] if t in w2ind.keys() else len(w2ind)+1 for t in tokens]
                vw.writerow([code] + [str(i) for i in inds])
                dw.writerow([code] + [t for t in tokens])


In [ ]:
SAVE_DIR = "./processed_data"
vocab_index_descriptions_ref('%s/vocab.csv' % SAVE_DIR,
                             '%s/description_words.vocab' % SAVE_DIR,
                             '%s/description_vectors.vocab' % SAVE_DIR)

In [ ]:
import os
from os.path import expanduser
import requests
import shutil
import torch
import csv
from collections import defaultdict

SAVE_DIR = "./processed_data"
BIO_BERT_LINK = "https://www.dropbox.com/s/dc2ki2d4jv8isrb/biobert_weights.zip?dl=1"
MODEL_SAVE_DICT = "./trained_models/"
TRAIN_DATA_PATH = os.path.join(SAVE_DIR, 'train_full.csv') # change this

def download_and_extract(tgt_dir):
    os.makedirs(tgt_dir, exist_ok=True)
    response = requests.get(BIO_BERT_LINK, stream=True)
    fp = os.path.join(tgt_dir, "biobert_weights.zip")
    if response.status_code == 200:
        with open(fp, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"download success: {fp}")
    else:
        print(f"download error: {response.status_code}")
    shutil.unpack_archive(fp, tgt_dir)
    # os.remove(fp)

def load_desc_map(desc_word_file):
    c2desc_map = {}
    with open(desc_word_file, 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = row[0]
            desc = ' '.join(row[1:-1])
            c2desc_map[code] = desc
    return c2desc_map

def buildc2idx(train_path):
    codes = set()
    for split in ['train', 'dev', 'test']:
        with open(train_path.replace('train', split), 'r') as f:
            lr = csv.reader(f)
            next(lr)
            for row in lr:
                for code in row[3].split(';'):
                    codes.add(code)
    codes = set([c for c in codes if c != ''])
    ind2c = defaultdict(str, {i:c for i,c in enumerate(sorted(codes))})
    c2ind = {c:i for i,c in ind2c.items()}
    return c2ind, ind2c

def code_desc_biobert():
    from transformers import BertConfig, BertModel, BertTokenizer
    model_dir = os.path.join(MODEL_SAVE_DICT, "biobert_v1.1_pubmed")
    assert os.path.exists(model_dir)

    model = BertModel.from_pretrained(model_dir)
    tokenizer = BertTokenizer.from_pretrained(model_dir)
    # tokenizer = BertTokenizer(vocab_file=voc_dir, do_lower_case=False)
    outputs = []

    c2desc = load_desc_map(os.path.join(SAVE_DIR, 'description_words.vocab'))
    c2ind, ind2c = buildc2idx(TRAIN_DATA_PATH)
    idx = 1
    for (cidx, code) in ind2c.items():
        desc = c2desc[code]
        inputs = tokenizer(desc, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            out = model(**inputs)
            embedding = outputs.last_hidden_state
            print(embedding.shape)
        idx += 1
        if idx > 2:
            break


In [ ]:
# download_and_extract(MODEL_SAVE_DICT)
code_desc_biobert()

In [ ]:
Y = 50
#first calculate the top k
counts = Counter()
dfnl = pd.read_csv('%s/notes_labeled.csv' % SAVE_DIR)
for row in dfnl.itertuples():
    for label in str(row[4]).split(';'):
        counts[label] += 1
codes_50 = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
codes_50 = [code[0] for code in codes_50[:Y]]
with open('%s/TOP_%s_CODES.csv' % (SAVE_DIR, str(Y)), 'w') as of:
    w = csv.writer(of)
    for code in codes_50:
        w.writerow([code])


In [ ]:
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('%s/%s_50_hadm_ids.csv' % (MIMIC_3_DIR, splt), 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('%s/notes_labeled.csv' % SAVE_DIR, 'r') as f:
        with open('%s/%s_%s.csv' % (SAVE_DIR, splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue
                codes = set(str(row[3]).split(';'))
                filtered_codes = codes.intersection(set(codes_50))
                if len(filtered_codes) > 0:
                    w.writerow(row[:3] + [';'.join(filtered_codes)])
                    i += 1

In [ ]:
for splt in ['train', 'dev', 'test']:
    filename = '%s/%s_%s.csv' % (SAVE_DIR, splt, str(Y))
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_%s.csv' % (SAVE_DIR, splt, str(Y)), index=False)